In [13]:
import requests
import json
import pandas as pd
import datetime
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('user', 'somepwd')

SERVERURL = "http://146.185.167.10/resampled/"

datatypes = ["fluorescens", "temp", "salt", "oxygene",  "turbidity"]
datatype = 'oxygene'

# get the data from the server
r = requests.get(SERVERURL+datatype+".json")

# make the JSON data native Python datastructure
d = json.loads(r.text)

# view one of the dive data
print(d[0])

{'ts': 'Tue, 12 May 2015 15:00:00 GMT', 'datatype': 'oxygene', 'divedata': [{'pressure(dBAR)': 18.5, 'oxygene': nan}, {'pressure(dBAR)': 12.5, 'oxygene': nan}, {'pressure(dBAR)': 3.5, 'oxygene': 100.46151201410002}, {'pressure(dBAR)': 1.5, 'oxygene': 99.46495734093332}, {'pressure(dBAR)': 17.5, 'oxygene': nan}, {'pressure(dBAR)': 10.5, 'oxygene': nan}, {'pressure(dBAR)': 9.5, 'oxygene': nan}, {'pressure(dBAR)': 4.5, 'oxygene': 100.24824826106665}, {'pressure(dBAR)': 11.5, 'oxygene': nan}, {'pressure(dBAR)': 0.5, 'oxygene': 96.05624543946665}, {'pressure(dBAR)': 13.5, 'oxygene': nan}, {'pressure(dBAR)': 14.5, 'oxygene': nan}, {'pressure(dBAR)': 2.5, 'oxygene': 100.0924995905}, {'pressure(dBAR)': 7.5, 'oxygene': 99.13400850779999}, {'pressure(dBAR)': 6.5, 'oxygene': 99.71782531193334}, {'pressure(dBAR)': 16.5, 'oxygene': nan}, {'pressure(dBAR)': 5.5, 'oxygene': 100.0189472512}, {'pressure(dBAR)': 8.5, 'oxygene': 99.22914901523332}, {'pressure(dBAR)': 15.5, 'oxygene': nan}], 'timeframe': 

In [14]:
# make a N*M matrix of the data and make sure the number of bins are OK
# we assume that all dives have a measurement that begins at 0.5 - if this is not true we need some rewrite              
y= []
x= []
tempz = []

# create the matrix and lists for x and y labels
for i in d:
    #print(i['ts'], type(i['ts']))
    # get a datetime object!
    ts = datetime.datetime.strptime(i['ts'], '%a, %d %b %Y %H:%M:%S GMT')
    x.append(ts) # 
    #datetime.datetime.fromtimestamp(i['ts'])) #['$date']/1000))
    # from datetime import datetime
    # date_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')
    col = []

    # sort the list so the dictionaries are sorted according to pressure
    newlist = sorted(i['divedata'], key=lambda k: -k['pressure(dBAR)'])
    for j in newlist:
        col.append(j[datatype])
        if -j['pressure(dBAR)'] not in y:
            y.append(-j['pressure(dBAR)'])

    tempz.append(col)
    #x.append(i['ts'])
z = list(map(list, zip(*tempz)))
y = sorted(y)

# tell plotly what to plot!
data=[
    go.Heatmap(
        z=z,
        x=x,
        y=y,
        type='surface'
        ),
    ]

# go do it!
py.iplot(data, filename='oxygene-surface')